In [1]:
# Set working directory
%cd "C:\Users\Patrick_Zager\coloc\data\raw\test"

# Import packages and modules
import glob
from tqdm import tqdm
import memory_profiler
import numpy as np
import pandas as pd
import scipy as sp
import skimage
import matplotlib.pyplot as plt

import math
from numpy import (mean, float_, dot, interp, uint8, uint16, 
                   uint64, log10, any as np_any, all as np_all)
from skimage import io
from skimage import filters
from skimage.filters import gaussian

C1,C2,C3 = 0,1,2
channels  = (C1,C2,C3)

def imread_rgb(f):
    return skimage.io.imread(f)

def manders(imagestack):
    
    fc = {} # Dictionary of fluorescent channels
    fc[C1] = imagestack[:,:,:,C1] # Indexing channels of images in image stack
    fc[C2] = imagestack[:,:,:,C2]
    fc[C3] = imagestack[:,:,:,C3]
    
    conditional = [] # Pre-define lists
    coloc1sums = []
    coloc2sums = []
    sums = []
    
    images = np.arange(len(imagestack)) # Evenly spaced array of integers with length equal 
                                        # to the number of images in the stack
    
    # Dictionary of functions callable for each channel
    for c in range (0, len(channels)): # Creates a boolean array classifying zeros and non-zeros for all channels
        cond = dict((i, fc[c][i] > 0) for i in images)
        conditional.append(cond)
    
    for i1, i2 in ((C1,C2), (C2,C3), (C3,C1)): # Array summation of the colocalization term
        coloc1 = dict((i, (fc[i1][i][conditional[i2][i]]).sum()) for i in images)
        coloc2 = dict((i, (fc[i2][i][conditional[i1][i]]).sum()) for i in images)
        coloc1sums.append(coloc1)
        coloc2sums.append(coloc2)
    
    for c in range(0, len(channels)): # Simple array summation of intensity for all channels
        summ = dict((i, fc[c][i].sum()) for i in images)
        sums.append(summ)
    
    result = {}
    
    for i1, i2 in ((C1,C2), (C2,C3), (C3,C1)): # Index through channels for all permutations of size two
        
        # Sum colocalization and and intensity terms for all images in the stack
        m1 = sum(coloc1sums[i1].values()) / sum(sums[i1].values())
        m2 = sum(coloc2sums[i1].values()) / sum(sums[i2].values())
        
        if i1 == 0:
            result["C1C2"] = m1
            result["C2C1"] = m2
        if i1 == 1:
            result["C2C3"] = m1
            result["C3C2"] = m2
        if i1 == 2:
            result["C3C1"] = m1
            result["C1C3"] = m2
    
    return result["C1C2"], result["C2C1"], result["C2C3"], result["C3C2"], result["C3C1"], result["C1C3"]

def maxers(imagestack):
    fc = {} # Dictionary of fluorescent channels
    fc[C1] = imagestack[:,:,:,C1] # Indexing channels of images in image stack
    fc[C2] = imagestack[:,:,:,C2]
    fc[C3] = imagestack[:,:,:,C3]
    
    maxs = [] # Pre-define list
    
    images = np.arange(len(imagestack)) # Evenly spaced array of integers with length equal 
                                        # to the number of images in the stack
    
    for c in range(0, len(channels)): # Simple array summation of intensity for all channels
        maxx = list((fc[c][i].sum()) for i in images)
        maxs.append(maxx)
        
    nmax = np.asarray(maxs) # convert list to numpy array
    
    result = list((np.argmax(nmax[c])) for c in channels) # Assemble list of indecies of maximum intensity images
    
    return result

%load_ext memory_profiler

[WinError 3] The system cannot find the path specified: 'C:\\Users\\Patrick_Zager\\coloc\\data\\raw\\test'
C:\Users\paz2009\CV\notebooks


In [2]:
# Read in data from .tif
exp_types = ['Exit', 'Return']
cond_types = ['CTRL','AKD','BKDD','ABKD']
time_types = ['t0', 't5', 't10', 't20', 't30', 't45']

data = skimage.io.ImageCollection('*.tif', conserve_memory=True, load_func=imread_rgb)
filenames = glob.glob('*.tif')

In [34]:
stacks = range(len(data))

data_blur = (gaussian(data[i]) for i in range(0, len(stacks)))

In [36]:
%%time
thresh_value = [skimage.filters.threshold_triangle(i[0,:,:,0]) for i in data_blur]

Wall time: 52.9 s


In [ ]:
thresh_list = []

for i in range(len(data_blur)):
    for c in range(len(channels)):
        thresh_value = skimage.filters.threshold_triangle(data_blur[i][thresh_index[i][c],:,:,c])
        thresh_list.append(thresh_value)
        low_values_flags = data[i][:,:,:,c] < thresh_value
        data_blur[i][:,:,:,c][low_values_flags] = 0
        
thresh_list = [thresh_list[i:i+len(channels)] for i in range(0, len(thresh_list), len(channels))]

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(18, 24))

ax[0, 0].imshow(data_blur[3][thresh_index[3][C1],:,:,C1], cmap='gray')
ax[0, 0].set_title('Image')

ax[0, 1].hist(data_blur[3][thresh_index[3][C1],:,:,C1].ravel(), bins=256)
ax[0, 1].set_title('Histogram')

ax[1, 0].imshow(data_blur[3][thresh_index[3][C2],:,:,C2], cmap='gray')
ax[1, 0].set_title('Image')

ax[1, 1].hist(data_blur[3][thresh_index[3][C2],:,:,C2].ravel(), bins=256)
ax[1, 1].set_title('Histogram')

ax[2, 0].imshow(data_blur[3][thresh_index[3][C3],:,:,C3], cmap='gray')
ax[2, 0].set_title('Image')

ax[2, 1].hist(data_blur[3][thresh_index[3][C3],:,:,C3].ravel(), bins=256)
ax[2, 1].set_title('Histogram')

In [ ]:
final = list((manders(data[i])) for i in tqdm(stacks))

In [ ]:
export = pd.DataFrame(final)
thresholders = pd.DataFrame(thresh_index)
filenames = pd.DataFrame(glob.glob('t*.tif'))
thresholdering = pd.DataFrame(thresh_list)

writer = pd.ExcelWriter('Ctrl_Exit_2.xlsx')
export.to_excel(writer, 'Sheet1')
thresholders.to_excel(writer, 'Sheet2')
filenames.to_excel(writer, 'Sheet3')
thresholdering.to_excel(writer, 'Sheet4')
writer.save()